# Predict climate from berry fungal communities

> using `ritme` (https://github.com/adamovanja/ritme)

note. use the **ritme_model** environment! 


In [10]:
from ritme.find_best_model_config import (
    _load_experiment_config,
    _load_phylogeny,
    _load_taxonomy,
    find_best_model_config,
)
from ritme.split_train_test import _load_data, split_train_test
from ritme.evaluate_tuned_models import evaluate_tuned_models

import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
workdir = '/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/climate-berries'
%cd $workdir

/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/climate-berries


In [12]:
# set experiment configuration path - this is downloaded from the github repo
# here we specify the column to be stratified by and the target! 
model_config_path = "/home/lfloerl/microterroir/Microbiome/Other_scripts/config/r_local_linreg_py.json"

# define path to feature table, metadata, and taxonomy (no phylogeny)
path_to_ft = "climate_filtered_table.qza"
path_to_md = "/home/lfloerl/microterroir/Microbiome/Metadata/ITS_Lavaux_Climate.tsv"
path_to_tax = "/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/taxonomy.qza"

# define train size
train_size = 0.8

In [13]:
config = _load_experiment_config(model_config_path)

In [14]:
md_df = pd.read_csv(path_to_md, sep='\t')
md_df.head()

,id,SAMPLE_NAME,sample_type,Plot_ID,COLLECTION_DATE,Year,Year_Cat,Fermentation_Timepoint,Plot,Altitude,...,maximum_rh,minimum_rh,cv_rh,GDD,average_temperature,median_temperature,maximum_temperature,minimum_temperature,accumulated_temperature,cv_temperature
0,364526_290-LP3-ITS-0866,Lavaux_2021-08-31_bark_Plot4,bark,Lavaux_Plot_4,2021-08-31 00:00:00 +0200,2021,Year2021,NaN,4,450,...,99.036583,33.078261,16.567322,1325.721843,18.154261,18.650958,26.689636,8.401750,2940.990343,22.587667
1,364526_289-LP3-ITS-0865,Lavaux_2021-08-31_bark_Plot5,bark,Lavaux_Plot_5,2021-08-31 00:00:00 +0200,2021,Year2021,NaN,5,400,...,95.964333,32.830458,16.331714,1379.347141,18.494554,19.047366,26.351409,8.816375,2996.117777,22.104867
2,364526_285-LP3-ITS-0861,Lavaux_2021-08-31_bark_Plot9,bark,Lavaux_Plot_9,2021-08-31 00:00:00 +0200,2021,Year2021,NaN,9,520,...,98.181542,32.794125,16.570451,1286.254021,17.902585,18.256629,25.850773,8.052083,2900.218734,22.831881
3,364526_287-LP3-ITS-0863,Lavaux_2021-08-31_bark_Plot11,bark,Lavaux_Plot_11,2021-08-31 00:00:00 +0200,2021,Year2021,NaN,11,490,...,96.437167,32.357792,16.118351,1297.271087,17.964399,18.541438,26.775286,7.966542,2910.232671,23.246939
4,364526_282-LP3-ITS-0858,Lavaux_2021-08-31_bark_Plot12,bark,Lavaux_Plot_12,2021-08-31 00:00:00 +0200,2021,Year2021,NaN,12,520,...,97.113875,32.104636,16.661423,1286.822875,17.897069,18.244984,27.509143,7.779875,2899.325119,23.615955


## Read & split data


In [15]:
md, ft = _load_data(path_to_md, path_to_ft)
print(md.shape, ft.shape)

(595, 36) (326, 6091)


In [16]:
train_val, test = split_train_test(
    md,
    ft,
    stratify_by_column=config["stratify_by_column"],
    feature_prefix=config["feature_prefix"],
    train_size=train_size,
    seed=config["seed_data"],
)

Train: (221, 6127), Test: (105, 6127)


/scratch/lfloerl/.condaenvs/ritme_model/lib/python3.10/site-packages/ritme/split_train_test.py:135: UserWarning: Provided feature table contains absolute instead of relative abundances. Hence, converting it to relative abundances...
  warnings.warn(


## Find best model config


In [17]:
tax = _load_taxonomy(path_to_tax)

best_model_dict, path_to_exp = find_best_model_config(
    config, train_val, tax, path_store_model_logs="ritme_all_models")

/scratch/lfloerl/.condaenvs/ritme_model/lib/python3.10/site-packages/optuna/distributions.py:708: UserWarning: The distribution is specified by [1, 20] and step=5, but the range is not divisible by `step`. It will be replaced by [1, 16].
  warnings.warn(
/scratch/lfloerl/.condaenvs/ritme_model/lib/python3.10/site-packages/optuna/distributions.py:708: UserWarning: The distribution is specified by [1, 10] and step=5, but the range is not divisible by `step`. It will be replaced by [1, 6].
  warnings.warn(
(train_nn_reg pid=3412050) Seed set to 12


(train_nn_reg pid=3412050) Train: (156, 66), Test: (65, 66)


(train_nn_reg pid=3412050) GPU available: False, used: False
(train_nn_reg pid=3412050) TPU available: False, using: 0 TPU cores
(train_nn_reg pid=3412050) HPU available: False, using: 0 HPUs
(train_nn_reg pid=3412050) /scratch/lfloerl/.condaenvs/ritme_model/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /scratch/lfloerl/tmpdata/ray/session_2025-02-12_10-05-50_883839_3395825/artifacts/2025-02-12_10-27-29/nn_reg/driver_artifacts/train_nn_reg_fdd33f93_1_batch_size=32,data_aggregation=None,data_selection=variance_threshold,data_selection_t=0.0001,data_transfo_2025-02-12_10-27-30 exists and is not empty.
(train_nn_reg pid=3412050) 
(train_nn_reg pid=3412050)   | Name   | Type       | Params | Mode 
(train_nn_reg pid=3412050) ----------------------------------------------
(train_nn_reg pid=3412050) 0 | layers | ModuleList | 1.1 K  | train
(train_nn_reg pid=3412050) ----------------------------------------------
(train_nn_reg pid=341205

(train_nn_reg pid=3412110) Train: (156, 42), Test: (65, 42)


(train_nn_reg pid=3412110) GPU available: False, used: False
(train_nn_reg pid=3412110) TPU available: False, using: 0 TPU cores
(train_nn_reg pid=3412110) HPU available: False, using: 0 HPUs
(train_nn_reg pid=3412110) /scratch/lfloerl/.condaenvs/ritme_model/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /scratch/lfloerl/tmpdata/ray/session_2025-02-12_10-05-50_883839_3395825/artifacts/2025-02-12_10-27-29/nn_reg/driver_artifacts/train_nn_reg_b5df96e2_2_batch_size=32,data_aggregation=None,data_selection=variance_threshold,data_selection_t=0.0023,data_transfo_2025-02-12_10-27-36 exists and is not empty.
(train_nn_reg pid=3412110) 
(train_nn_reg pid=3412110)   | Name   | Type       | Params | Mode 
(train_nn_reg pid=3412110) ----------------------------------------------
(train_nn_reg pid=3412110) 0 | layers | ModuleList | 289    | train
(train_nn_reg pid=3412110) ----------------------------------------------
(train_nn_reg pid=341211

...

(train_nn_reg pid=3464813) Train: (156, 159), Test: (65, 159)


(train_nn_reg pid=3464813) GPU available: False, used: False
(train_nn_reg pid=3464813) TPU available: False, using: 0 TPU cores
(train_nn_reg pid=3464813) HPU available: False, using: 0 HPUs
(train_nn_reg pid=3464813) /scratch/lfloerl/.condaenvs/ritme_model/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /scratch/lfloerl/tmpdata/ray/session_2025-02-12_10-05-50_883839_3395825/artifacts/2025-02-12_10-27-29/nn_reg/driver_artifacts/train_nn_reg_f9cf00e2_99_batch_size=32,data_aggregation=None,data_selection=abundance_ith,data_selection_i=2,data_transform=None,e_2025-02-12_10-55-27 exists and is not empty.
(train_nn_reg pid=3464813) 
(train_nn_reg pid=3464813)   | Name   | Type       | Params | Mode 
(train_nn_reg pid=3464813) ----------------------------------------------
(train_nn_reg pid=3464813) 0 | layers | ModuleList | 1.7 M  | train
(train_nn_reg pid=3464813) ----------------------------------------------
(train_nn_reg pid=346481

(train_nn_reg pid=3464960) Train: (156, 537), Test: (65, 537)


(train_nn_reg pid=3464960) GPU available: False, used: False
(train_nn_reg pid=3464960) TPU available: False, using: 0 TPU cores
(train_nn_reg pid=3464960) HPU available: False, using: 0 HPUs
(train_nn_reg pid=3464960) /scratch/lfloerl/.condaenvs/ritme_model/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /scratch/lfloerl/tmpdata/ray/session_2025-02-12_10-05-50_883839_3395825/artifacts/2025-02-12_10-27-29/nn_reg/driver_artifacts/train_nn_reg_de26a40b_100_batch_size=32,data_aggregation=None,data_selection=abundance_ith,data_selection_i=3,data_transform=None,_2025-02-12_10-55-34 exists and is not empty.
(train_nn_reg pid=3464960) 
(train_nn_reg pid=3464960)   | Name   | Type       | Params | Mode 
(train_nn_reg pid=3464960) ----------------------------------------------
(train_nn_reg pid=3464960) 0 | layers | ModuleList | 1.2 M  | train
(train_nn_reg pid=3464960) ----------------------------------------------
(train_nn_reg pid=346496

RuntimeError: Some trials encountered errors. See above for reported Ray Tune errors.

## Evaluate best models


In [18]:
metrics = evaluate_tuned_models(best_model_dict, config, train_val, test)
metrics

NameError: name 'best_model_dict' is not defined